# Capstone Project Sandbox

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/Users/ssrosa/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [64]:
import copy

In [160]:
class Lot(object):
    def __init__(self):
        self.size = None
        self.zoned_as = None
        #Coords where lot is. Just for viz
        self.loc = []
        self.residences = [[]]

class Developer(object):
    def __init__(self):
        self.residences_built = [[]]
        #100sqft - about a 9X12 room
        #Hard coded for now
        self.min_res_size = 100
    def build_on(self, lot):
        #Remaining empty land on this lot:
        avail_area = lot.size - sum([residence.size for residence in lot.residences[-1]])
        #If avail area exceeds min residence size that this developer can build
        if  avail_area > self.min_res_size:
            #Distribution of possible sizes for a new residence
            size_dist = np.arange(self.min_res_size, avail_area)
            #Initialize new residence object
            #Give residence a random size within the realm of possibility
            size = np.random.choice(size_dist)
            residence = Residence(size, lot.zoned_as)
            #Add residence to last list of residences built on the lot
            lot.residences[-1].append(residence)
            #Add residence to last list of residence built by this developer
            self.residences_built[-1].append(residence)

class Residence(object):
    def __init__(self, size, zoned_as):
        self.size = size
        self.zoned_as = zoned_as
        #100sqft - about a 9X12 room
        #Hard coded for now
        self.min_unit_size = 100
        
        max_units = self.zoned_as // 8 #8 people per unit
        # if zoned_as 8, can have 1 unit
        # if zoned_as 16, can have 2 units
        # if zoned_as 48, cah nave 6 units
        # if zoned as 96, can have 12 units
        self.units = [[]]
        #Not clear at this point whether nested list will be necessary but am including it for now.
        max_units = zoned_as / 8
        #Units can be larger than 100 sqft
        if size / max_units >= self.min_unit_size:
            self.unit_size = size / max_units
        #But cannot be smaller
        else:
            self.unit_size = self.min_unit_size
        avail_area = self.size
        while  avail_area >= self.unit_size:
            #Create a new unit object
            unit = Unit(self.unit_size)
            #Add unit object to last item of Residence's units list.
            self.units[-1].append(unit)
            avail_area -= self.unit_size

    #def demolish():
    #check that all residents are moved out from units
    #remove residence from lot

class Unit(object):
    def __init__(self, size):
        self.size = size
        #Value is a list. -1 is current value.
        self.value = [(size * price_sqft)]
        #Occupant is a list. -1 is current occupant.
        self.occ = [False]
    def update(self):
        self.value.append(self.value[-1])
        self.occ.append(self.occ[-1])

class Household(object):
    '''
    '''
    def __init__(self, has):
        #List of whether the household is homeless or not
        self.housed = [False]
        #Hold income constant for now but still write as a list
        self.has = [has]
    
    def update(self):
        self.housed.append(self.housed[-1])
        self.has.append(self.has[-1])
    
    def can_move_to(self, unit):
        #household needs a place to live
        #unit not already taken
        # household can afford unit 
        if (not self.housed[-1]) \
        and (not unit.occ[-1]) \
        and (unit.value[-1] <= self.has[-1]):        
            return True
        else: 
            return False
    
    def move_to(self, unit):
        '''
        Extra checks to make sure household doesn't get assigned taken unit or mispriced unit.
        Changes .housed[-1] to True and adds this household to the list of occupants in the unit.
        '''
        #Household becomes occupant of unit
        unit.occ[-1] = self
        #Houshold gets marked as housed
        self.housed[-1] = unit
        #print(f'Household {self.has[-1]} now lives in unit {unit.value[-1]}')
        
def create_lots(self, land, lot_size_avg, lot_size_std, zon_avg, zon_std):
    new_lots = []
    #Create a distribution to simulate variety of lot sizes
    lot_dist = np.random.normal(scale = lot_size_std, loc = lot_size_avg, size = num_lots)
    #Create a distribution to simulate variety of zoning
    #Empty for now
    
    while land > 0:
        size = np.round(np.random.choice(lot_dist, replace = False), 0)
        #Create a new lot only if the lot size will fit on the available land left
        if size > land:
            #Need to break the loop here otherwise it keeps running until a random size is
            #drawn that fits the available remaining land.
            break
        else:
            #Instantiate a lot with a random size
            lot = Lot()
            lot.size = size
            #Reduce available land area by the size of this lot
            land -= size
            #Randomly zone the lot
            lot.zoned_as = np.random.choice([8, 16, 48, 96], replace = True)
            #Add this lot to the list of lots in the sim
            new_lots.append(lot)
    #Append this list to the list of lots as the last item        
    self.lots.append(new_lots)
        
def create_developers(self):
    '''
    Instantiate some number of developer objects.
    '''
    developer = [Developer()]
    self.developers.append(developer)
                
def round_of_developing(self, developers, lots):
    '''
    Iterate over list of developers and call each developer's develop method
    '''
    [developer.build_on(lot) for lot in lots for developer in developers]

def create_households(self, pop_growth, has_avg, has_std):
    '''
    Create some number of households to come to the city.
    '''
    income_dist = np.random.normal(scale = has_std, loc = has_avg, size = pop_growth)
    self.households[-1].extend([Household(i) for i in income_dist])
    
def round_of_moving(self):
    '''
    Iterate over list of households and call each household's move method.
    '''

    #Prepare units and households for moving.
    #Step 1: Fet units available.
    ua = sorted([u for u in self.units[-1] if not u.occ[-1]], \
                        key = lambda u: u.value[-1], reverse = True)
    #Step 2: Get households seeking unit.
    hh = sorted([h for h in self.households[-1] if not h.housed[-1]], \
                        key = lambda h: h.has[-1], reverse = True)
    #Step 3: Exclude any units with value above the max spending power 
    #of the households. (Wouldn't be able to rent to anyone)
    ua = [u for u in ua if u.value[-1] <= hh[0].has[-1]]
    #Step 4: Exclude any households with spending power less than 
    #the lowest unit value. (Wouldn't be able to find a place.)
    #Keep hh same length as or shorter than ua.
    #(No unit can take more than household so there cannot be more hh than ua.)
    hh = [h for i, h in enumerate(hh) if (h.has[-1] >= ua[-1].value[-1]) and \
                                                                    (i < len(ua))]
    #Step 5: call the match function to move households into units.
    match(hh, ua)
    
    #These two numbers should be equal
    num_hh_moved = sum([1 if h.housed[-1] else 0 for h in hh])
    num_ua_moved_into = sum([1 if u.occ[-1] else 0 for u in ua])

def match(hh_, ua):
    '''
    Helper function for round of moving. Does the work of matching 
    household to unit. Iterates over units, trying to match each 
    to the highest-value household in the list. If there's a match,
    the household gets popped and the loop moves on to next unit.
    If there's not a match, the loop just moves on to the next unit. 
    A household can take a unit that ncosts less than or equal to
    its spending power. Some rich households may wind up getting 
    cheap units. Units may go untaken; households may go unhoused.

    '''
    #New list to avoid popping the original
    hh = [h for h in hh_]
    #For each unit in list 
    for u in ua:
        #If there are households left in the list
        if hh:
            #If household can take the unit
            if hh[0].can_move_to(u):
                #Household gets unit
                hh[0].move_to(u)
                #Household gets removed from list
                hh.pop(0)
    
def print_stats(self, time_step):
    print(f'Population at time step {time_step}: {len(self.households[-1])}')
    print(f'{len(self.lots[-1])} lots on {land} sqft of land')
    print(f'{len(self.developers)} developers')
    #Get number of residences. FOr now there's no master list for some reason
    num_residences = len(self.residences[-1])
    num_units = len(self.units[-1])
    print(f'{num_residences} residences and {num_units} units')
    housing = [True if household.housed[-1] else False for household in self.households[-1]]
    num_housed = sum(housing)
    num_homeless = len(housing) - num_housed
    print(f'{num_housed} households have housing and {num_homeless} households are homeless.')
    
class Simulation(object):
    def __init__(self, 
                 #For creating lots
                 land, num_lots, lot_size_avg, lot_size_std, zon_avg, zon_std,
                 #For bringing people to the city
                 pop_growth, has_avg, has_std, 
                 #For running the sim over time
                 total_time_steps, inflation):
        #Number of years to run sim
        self.total_time_steps = total_time_steps
        #List of increments by which to increase population
        self.pop_growth = pop_growth
        #Held as constant for now
        self.has_avg = has_avg
        #Held as constant for now
        self.has_std = has_std
        #Held as constant for now
        self.inflation = inflation
        
        #STEP 1: 
        #Fill up land area with lots
        self.lots = []
        #Will return a list of lots. Becomes lots[-1]
        self.create_lots(land, lot_size_avg, lot_size_std, zon_avg, zon_std)
       
        #STEP 2: Create developers to build residences
        self.developers = []
        self.create_developers()
        self.round_of_developing(self.developers[-1], self.lots[-1])
        
        #Store residences in a list
        self.residences = [[residence for lot in self.lots[-1] for residence in lot.residences[-1]]]
        #Store units in a list
        self.units = [[unit for residence in self.residences[-1] for unit in residence.units[-1]]]
        
        #STEP 3: Create people to come to the city and move in
        self.households = [[]]
        #Pop the last value of population growth
        self.create_households(self.pop_growth.pop(), 
                               self.has_avg, 
                               self.has_std)
        
        #STEP 4: Let each household try to move into a house
       # self.round_of_moving()
        
        #STEP 5: Print initial statistics
        print('Simulation started.')
        self.print_stats(0)
        
def time_step(self, step):
    
    #Demolish any old residences
    #pass
    #Kill off any old households
    #pass
    print('\n')
    print('Updating object records')
    print('Lots:')
    #Update all lists of objects
    self.lots.append(self.lots[-1])
    print('Developers:')
    self.developers.append(self.developers[-1])
    #self.residences.append(self.residences[-1])
    #self.units.append(self.units[-1])
    #Update each existing unit
    print('Updating each unit')
    [u.update() for u in self.units[-1]]
    print('Updating list of units')
    self.households.append(self.households[-1])
    #Update each existing household
    print('Updating each household')
    [h.update() for h in self.households[-1]]
    
    #Inflation increases values or something
    
    #New developers
    #Leave out for now
    #Will need to extend [-1] spot of developer list with any new developers
    
    #New households move to town
    #Note to self: this should automatically extend the [-1] list of households
    print('adding households')
    self.create_households(self.pop_growth.pop(), self.has_avg, self.has_std)
    print('building new residences')
    #Build new residences on available land
    self.round_of_developing(self.developers[-1], self.lots[-1])
    #Manually append the list of all residences
    #Not recording residences year by year for now
    #althoguh they could be retrieved from the history in each developer
    print('appending new residences to master list')
    self.residences.append([residence for lot in self.lots[-1] for residence in lot.residences[-1]])
    #Need to manually add units to list for sim
    print('appending new units to master list')
    self.units.append([unit for residence in self.residences[-1] for unit in residence.units[-1]])
    #Have homeless try to move into new places
    print('people moving')
    self.round_of_moving()
    
def run(self):
    pass
    for _ in tqdm(range(self.total_time_steps)):
        # Print out the current time step 
        print(f"Time step {_}")
        self.time_step(_)
        self.print_stats(_)

In [162]:
#Attach all these functions to the Simulation object
Simulation.create_lots = create_lots
Simulation.create_developers = create_developers
Simulation.create_households = create_households
Simulation.round_of_developing = round_of_developing        
Simulation.round_of_moving = round_of_moving
Simulation.print_stats = print_stats
Simulation.time_step = time_step
Simulation.run = run

#CITY SETTINGS
#Size of land area in sqft (for now)
land = 1000000000 #one trillion sqft in SF
num_lots = 10 #800,000 in SF
lot_size_avg = 5000
lot_size_std = 50
zon_avg = None
zon_std = None


#Starting population
pop_growth = [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 100000]
#[1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 1000,]
            # 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 1000]
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
#[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 100]
             #,
             ##1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 1000,
             #1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 1000]
has_avg = 1500
has_std = 500

#Time and money
price_sqft = 1
inflation = 0.02
years = 10

#CREATE SIM
%%time
sim = Simulation(land = land, 
                 num_lots = num_lots, 
                 lot_size_avg = lot_size_avg, 
                 lot_size_std = lot_size_std,
                 zon_avg = zon_avg, 
                 zon_std = zon_std,
                 pop_growth = pop_growth, 
                 has_avg = has_avg, 
                 has_std = has_std, 
                 total_time_steps = years,
                 inflation = inflation
                 )

Simulation started.
Population at time step 0: 100000
199525 lots on 1000000000 sqft of land
1 developers
199525 residences and 937557 units
0 households have housing and 100000 households are homeless.


In [163]:
%%time
sim.run()

Time step 0


Updating object records
Lots:
Developers:
Updating each unit
Updating list of units
Updating each household
adding households
building new residences
appending new residences to master list
appending new units to master list
people moving
Population at time step 0: 110000
199525 lots on 1000000000 sqft of land
2 developers
394937 residences and 1690003 units
109707 households have housing and 293 households are homeless.
Time step 1


Updating object records
Lots:
Developers:
Updating each unit
Updating list of units
Updating each household
adding households
building new residences
appending new residences to master list
appending new units to master list
people moving
Population at time step 1: 120000
199525 lots on 1000000000 sqft of land
3 developers
570568 residences and 2210229 units
119684 households have housing and 316 households are homeless.
Time step 2


Updating object records
Lots:
Developers:
Updating each unit
Updating list of units
Updating each household


# RESUME HERE. Sim is functional with 2.7m units and 200k households. Take 1.5mins.

In [129]:
len(new_hh)

34

In [131]:
len(hh)

4507

In [98]:
sim.households[-1][2].move_to(sim.units[-1][0])

In [99]:
vars(sim.households[-1][2])

{'housed': [<__main__.Unit at 0x116f1d198>], 'has': [1309.2095705503789]}

In [132]:
sum([1 if h.housed[-1] else 0 for h in sim.households[-1]])

4473

In [107]:
sum([1 if u.occ[-1] else 0 for u in sim.units[-1]])

4805

In [280]:
#Testing for round of moving and match



In [ ]:
#TOY DATA for testing match method
# ua = np.array([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])
#hh = np.array([10.1, 10, 10, 4, 3, 1, 1, 1, 1, 1, 1])

# i = 0
# j = 0   
# match(hh, ua, i, j)

In [ ]:
import pandas as pd
pd.Series(unit_val_arr).value_counts()

In [ ]:
plt.hist([u.value[-1] for u in sim.units[-1]])

# Gather data for viz of sim run

In [ ]:
#lots

num_residences = [len(res) for res in sim.residences]

residence_size_avg = [(sum([r.size for r in res]) / len(res)) for res in sim.residences]

units_num = [len(uni) for uni in sim.units]

unit_size_avg = [(sum([u.size for u in uni]) / len(uni)) for uni in sim.units]

unit_value_avg = [(sum([u.value[-1] for u in uni]) / len(uni)) for uni in sim.units]

#Problem in the recording of households
households_num = [len(hou) for hou in sim.households]

households_housed = [len([h for h in hou if h.housed[]]) for hou in sim.households]

# households_homeless

#For now is not a list
# household_income_avg

In [ ]:
sim.households[0][0].housed

# RESUME HERE. Something with lambda to get only the housing bool that you want for a certain yeare

# VARIOUS CHECKS FOR OBJECTS IN SIM

In [ ]:
#Check sizes of all lots
#sum([lot.size for lot in sim.lots[-1]])

In [ ]:
#Check size of each residence
#[residence.size for lot in sim.lots[-1] for residence in lot.residences[-1]]

In [ ]:
#Check size of each unit
unit_sizes = [unit.value for lot in sim.lots[-1] for residence in lot.residences[-1] for unit in residence.units[-1]]

In [ ]:
len(unit_sizes)

In [ ]:
set(unit_sizes)

In [ ]:
plt.hist(unit_sizes, bins = 50)

In [ ]:
#Test for bin_items: look at all binned items
#[v.value[-1] for key, val in binned_items.items() for v in val]

#Look at one bin
#[v.value[-1] for v in binned_items[445.3666666666667]]

In [ ]:
#Check area of each residence that isn't carved into units
#[(residence.size - sum(unit.size for unit in residence.units[-1])) for lot in sim.lots[-1] for residence in lot.residences[-1]]

In [ ]:
#Check unbuilt land of each lot
#avail_area = [lot.size - sum([residence.size for residence in lot.residences[-1]]) for lot in sim.lots[-1]]
#Avail area on each lot should be less than min_size for building a new residence
#avail_area

In [ ]:
#Check how many residences on each lot
#[len(lot.residences[-1]) for lot in sim.lots[-1]]

In [ ]:
#Check how many units in each residence
#[len(residence.units[-1]) for lot in sim.lots[-1] for residence in lot.residences[-1]]

In [ ]:
#Check a single lot
#vars(sim.lots[-1][0])

In [ ]:
#Check a single residence on a lot
#vars(sim.lots[-1][0].residences[-1][0])

In [ ]:
#Check a single unit in a residence
#vars(sim.lots[-1][0].residences[-1][0].units[-1][0])

In [ ]:
#THIS IS GREAT CODE DON'T DELETE
#Check all unit values
#How to quadruple nest a list comprehension:
#smallest for biggest in biggest for second biggest in second biggest for third biggest in third biggest
unit_values = [unit.value[-1] for lot in sim.lots[-1] for residence in lot.residences[-1] for unit in residence.units[-1]]
#unit_values
#plt.hist(unit_values)

In [ ]:
#Get total number of units in all residences on all lots
len([unit.value[-1] for lot in sim.lots[-1] for residence in lot.residences[-1] for unit in residence.units[-1]])

In [ ]:
#Check incomes for all households
#incomes = [household.has[-1] for household in sim.households[-1]]
#plt.hist(incomes)

In [ ]:
#Check incomes for currently housed households
#incomes_currently_housed = [household.has[-1] for household in sim.households[-1] if household.housed[-1]]
#plt.hist(incomes_currently_housed)

In [ ]:
#Check housing history for all households
#housing_history = [household.housed for household in sim.households[-1]]
#housing_history

In [ ]:
# #Test for instantiation of lots, residences built by developers, units per residence
# for lot in sim.lots[-1]:
#     print('Lot size:', lot.size)
#     print('Number of buildings on lot:', len(lot.residences[-1]))
#     for residence in lot.residences[-1]:
#         print('Building size:', residence.size)
#         print('Building zoned as:', residence.zoned_as)
#         print('Units in building:', len(residence.units[-1]))
#         print('Sqft per unit:', residence.size / len(residence.units[-1]))
#         for unit in residence.units[-1]:
#             print('Unit size:', unit.size)
#             print('Unit initial value:', unit.value[0])
#             print('Unit current value:', unit.value[-1])
#             print('Household size:', unit.occ[-1].size)
#             print('Household spending power:', unit.occ[-1].size)
#     print('\n')

In [ ]:
#CITY SETTINGS
#Size of land area in sqft (for now)
land = 1000000 #one trillion sqft in SF
num_lots = 1000 #800,000 in SF
lot_size_avg = 2000
lot_size_std = 500

#Starting population
pop_start = 1000
has_avg = 2000
has_std = 500

#Time and money
inflation = 0.02
years = 10

#Probability distribution of house sizes
#house_size_dist = np.random.normal(scale = house_size_std, loc = avg_house_size, size = households)


sim = Simulation(total_time_steps = years, 
                 land = land, 
                 num_lots = num_lots, 
                 lot_size_avg = lot_size_avg, 
                 lot_size_std = lot_size_std,
                 pop_start = pop_start, 
                 has_avg = has_avg, 
                 has_std = has_std, 
                 inflation = inflation,
                 )

# Viz: Assigning lots at start of sim (throws an error when top of grid is reached)

In [ ]:
#VIZ
def draw_land_grid(land):
    '''Writes a dict of y values, each with a dict of x values.
    Works as a grid of xes and yes.'''
    root = int(np.round(np.sqrt(land), 0) + 3)
    x_range = root
    y_range = root
    #Each point in the dict will get switched True when a house is drawn on it
    grid = {y: {x: False for x in np.arange(x_range)} for y in np.arange(y_range)}
    return grid

def assign_loc(lot, x, y, frontage):
    '''
    Assign a location to a lot.
    '''
    xes = np.arange(x, x + frontage)
    yes = np.arange(y, y + frontage)
    loc = [(x, y) for y in yes for x in xes]
    lot.loc = loc
    
def fill_coords(grid, loc):
    '''
    Fill the coords in the grid dict after assignment of previous lot.
    '''
    for coord in loc:
        #Reset the point to True in the dict of grid
        grid[coord[1]][coord[0]] = True

def avail(grid, x_, y_, frontage):
    #X adjusted to the left for padding
    x = x_ - 1
    #Y adjusted to the left for padding
    y = y_ - 1
    x_range = np.arange((x), (x + frontage + 2))
    y_range = np.arange((y), (y + frontage + 2))
    #List of bools returned from each point checked
    bottom = [grid[y][x_i] for x_i in x_range[:-1]]
    top = [grid[y + frontage][x_i] for x_i in x_range[1:]]
    left = [grid[y_i][x] for y_i in y_range[1:]]
    right = [grid[y_i][x + frontage] for y_i in y_range[:-1]]
    
    #Value of all bools
    points = sum(bottom) + sum(top) + sum(left) + sum(right)
    
    #If even one point in the proposed lot is taken (points != 0)
    if points:
        return False
    #If not a single point in the proposed lot is taken (points == 0)
    else:
        #print(x, y, 'is avail!')
        return True

#(Each row is a tuple of (x, bool))
def find_loc(grid, lot, next_x, next_y):
    '''
    #Traverse each row in the land grid via grid dict
    '''
    #Padding -1 right side
    x_range = len(grid[0]) - 1
    #Padding -1 top
    y_range = len(grid) - 1
    
    #One side of the square lot
    frontage = int(np.round(np.sqrt(lot.size), 0))
    
     #Traverse the y rows in the grid dict
    for y in np.arange(next_y, y_range):
        #If row isn't too full for this lot
        if x_range - sum([v for k, v in grid[y].items()]) > frontage:
            #print('Looking in row', y)
            #Traverse the row of x values
            for x in np.arange(next_x, x_range):
                #print('Looking in x', x)
                #If the right side of the lot is at the right edge of the grid, 
                #and at or below the top, continue
                if ((x + frontage) <= (x_range - 1)) and ((y + frontage) <= (y_range - 1)):
                    #If all frontage points in the proposed lot return False
                    if avail(grid, x, y, frontage):
                        #print('Assigning lot at', x, y)
                        #House gets assigned these coords
                        assign_loc(lot, x, y, frontage)
                        #Coords get changed to True in the grid dict
                        fill_coords(grid, lot.loc)
                        #Break the loop
                        #print('Found lot for ', lot.size)
                        #Return next x and next y to start on
                        #print((x + frontage), y)
                        return (x + frontage), y
        #If spot isn't found, reset x to 0 before moving up to next y
        #(Start at 1, not 0, for padding on left side)
        next_x = 1


In [ ]:
#Draw grid for sim
grid = draw_land_grid(land)

counter = 0
#Padding +1
next_x = 1
#Padding +1
next_y = 1

for _ in tqdm(range(len(sim.housing_stock))):
    #Draw a lot on the grid
    next_x, next_y = find_loc(grid, sim.housing_stock[_], next_x, next_y)
    #counter += 1
    #print(counter, 'out of', len(sim.housing_stock))

In [ ]:
#What does the grid look like?
grid[0]

In [ ]:
sim.housing_stock[10].sold

In [ ]:
sim.run()

# Visualizing lots (some overlap)

In [ ]:
#Draw viz of all lots
fig = plt.figure(figsize = (14,14))
for lot in sim.housing_stock:
    if lot.sold:
        color = 'blue'
    else:
        color = np.random.choice(['dimgray', 'darkgray', 'silver', 'slategray', 'powderblue', 'whitesmoke'])
    x = [point[0] for point in lot.loc]
    y = [point[1] for point in lot.loc]
    #ADD IF STATEMENT FOR SALE OF LOT
    plt.scatter(x, y, label = lot.size, marker = 's', color = color)
plt.show()

In [ ]:
#Draw viz of all lots
fig = plt.figure(figsize = (14,14))
for lot in sim.housing_stock:
    if lot.sold:
        color = 'blue'
    else:
        color = np.random.choice(['dimgray', 'darkgray', 'silver', 'slategray', 'powderblue', 'whitesmoke'])
    x = [point[0] for point in lot.loc]
    y = [point[1] for point in lot.loc]
    #ADD IF STATEMENT FOR SALE OF LOT
    plt.scatter(x, y, label = lot.size, marker = 's', color = color)
plt.show()

In [ ]:
# def match(hh, ua, i, j, indices):
#     '''
#     Takes in numpy arrays of unit values and household spending power and
#     compares them to match units to households.
#     A household can take a unit that costs less than or equal to
#     its spending power but not more. 
#     (Some rich households may wind up getting cheap units.)
#     (Many poor household just won't get a unit.)
#     '''
#     #indices = indices
#     #COMBINE THESE FIRST THREE LATER AFTER SATISFIED WITH CODE.
#     #Case 1: ua index is out of bounds, stop the sequence.
#     if (j >= len(ua)):
#         print(f'Case 1: End of units reached: j is {j}')
#         print(f'len(hh) {len(hh)}, i at {i}, j at {j},')
#         print(f'and indices at {indices}')
#     #Case 2: hh index is out of bounds, stop the sequence.
#     elif (i >= len(hh)):
#         print(f'Case 2: End of households reached: i is {i} and len(hh) is {len(hh)}')
#         print(f'len(hh) {len(hh)}, i at {i}, j at {j},')
#         print(f'and indices at {indices}')
#     #Case 3: If the rest of the hh array is zeros, stop the sequence.
#     #(Possible if there are more households than units. Some go unhoused.)
#     elif (hh[i:].sum() == 0):
#         print(f'Case 3: Remaining households are zeros: {hh[i:-1].sum()}')
#         print(hh[i:-1])
#         print(f'len(hh) {len(hh)}, i at {i}, j at {j},')
#         print(f'and indices at {indices}')
#     #Case 2: continue the sequence.
#     else:
#         #Case A: If this h is a 0, just move down one row of h
#         #but stay on same row of units
#         if not hh[i]:
#             i += 1
#             print(f'Case A: hh at {i} is zero: {hh[i]}')
#         #Case B: If this h isn't zero and it's greater than/equal to the unit,
#         #then the household can move into the unit.
#         #Move down one row in both arrays.
#         elif hh[i] and hh[i] >= ua[j]:
#             print(f'Case B: {hh[i]} can live at{ua[j]}')
#             i += 1
#             j += 1
#         #Case C: If this h isn't zero and it's less than the unit,
#         #then no households can afford this unit. Move down one row in both arrays.
#         #(This unit at index j goes unoccupied.)
#         elif hh[i] and hh[i] < ua[j]:
#             print(f'Case C: {hh[i]} cannot live at {ua[j]}')
#             print(F'Inserting a zero into hh array before index {i}')
#             #print('New hh:')
#             #print(hh)
#             hh = np.insert(hh, i, 0)
#             indices.append(i)
#             i += 1
#             j += 1
#      #End of Case 2. Now recursively call match.   
#         print(f'Recursively calling match with len(hh) {len(hh)}, i at {i}, j at {j},')
#         print(f'and indices at {indices}')
#         match(hh, ua, i, j, indices)   
#     #Outside the loops, return indices to ride the layers of recursion
#     return indices